In [1]:
import requests
import time
import csv
import pandas as pd

In [2]:
etherscan_API_key="7QYHPEXQXE3JSWJI4CVGZDJVZ9ZDYH3F2Y"

def get_transfers_in_block_range(start_block, end_block, contract_address):
    transfers = []
    url = f"https://api.etherscan.io/api"
    
    params = {
        "module": "account",
        "action": "tokentx",
        "contractaddress": contract_address,
        "startBlock": start_block,
        "endBlock": end_block,
        "sort": "asc",
        "apikey": etherscan_API_key
    }
    
    response = requests.get(url, params=params)
    data = response.json()
    
    if 'result' in data:
        for transfer in data['result']:
            transfer_info = {
                'timestamp': transfer['timeStamp'],
                'value': transfer['value'],
                'from': transfer['from'],
                'to': transfer['to']
            }
            transfers.append(transfer_info)
    
    return transfers

In [3]:
def save_transfers_to_csv(transfers, filename):
    print("Saving")
    keys = transfers[0].keys()
    with open(filename, 'w', newline='') as output_file:
        dict_writer = csv.DictWriter(output_file, fieldnames=keys)
        dict_writer.writeheader()
        dict_writer.writerows(transfers)

In [4]:
def get_all_transfers_in_chunks(start_block, end_block, contract_address, block_chunk_size=10000):
    all_transfers = []
    current_block = start_block
    
    while current_block < end_block:
        next_block = min(current_block + block_chunk_size, end_block)
        print(f"Querying transfers from block {current_block} to {next_block}...")
        
        transfers = get_transfers_in_block_range(current_block, next_block, contract_address)
        all_transfers.extend(transfers)
        
        current_block = next_block + 1
    
    return all_transfers

In [12]:
# transfers = get_all_transfers_in_chunks(15412117, 20866918, "0x3Afdc9BCA9213A35503b077a6072F3D0d5AB0840", block_chunk_size=1000000)
transfers = get_transfers_in_block_range(15412117, 20866918, "0xA17581A9E3356d9A858b789D68B4d866e593aE94")
save_transfers_to_csv(transfers, "cWETHv3.csv")

Saving
